## COVID-19 Modeling Pipeline

In [ ]:
!bq --location=US mk --dataset covid_19_modeled

In [ ]:
%%bigquery
create or replace table covid_19_modeled.Cases
as select null as id, * from covid_19_staging.Cases

### Compute fingerprint of location fields (state, country) 

In [ ]:
%%bigquery
update covid_19_modeled.Cases set id = FARM_FINGERPRINT(country) 
where state is null

#### Task 1: Compute fingerprint of state + country where state is not null

#### Make sure that id field has no null values

In [ ]:
%%bigquery
select count(*) as null_id_count
from covid_19_modeled.Cases
where id is null

In [ ]:
%%bigquery
select id, state, country
from covid_19_modeled.Cases
order by state, country
limit 5

### Split Cases table

In [ ]:
%%bigquery
create or replace table covid_19_modeled.Location_Temp
as select distinct id, state, country, latitude, longitude
from covid_19_modeled.Cases

#### Task 2: Get record count from table Location_Temp

In [ ]:
%%bigquery
create or replace table covid_19_modeled.Event_Temp
as select id as location_id, last_update, confirmed, deaths, recovered
from covid_19_modeled.Cases

#### Task 3: Get record count from table Event_Temp

#### Sample oldest and most recent Event_Temp records

In [ ]:
%%bigquery
select * from covid_19_modeled.Event_Temp
order by last_update
limit 5

In [ ]:
%%bigquery
select * from covid_19_modeled.Event_Temp
order by last_update desc
limit 5

### Option A: Standardize timestamps with SQL

In [ ]:
%%bigquery
create or replace table covid_19_modeled.Event_SQL1 as
select *
from covid_19_modeled.Event_Temp
where strpos(last_update, '/') > 0

#### Note: Event_SQL1 has all the records with '/'

#### Task 4: Get record count for timestamps containing '/' (i.e. mm/dd/yyyy or mm/dd/yy):

In [ ]:
%%bigquery
create or replace table covid_19_modeled.Event_SQL2 as
select location_id, cast(last_update as datetime) last_update, confirmed, deaths, recovered 
from covid_19_modeled.Event_Temp
where strpos(last_update, '-') > 0

#### Note: Event_SQL2 has all the records with '-'

#### Q5: Get record count for timestamps containing '-' (i.e. yyyy-mm-dd):

In [ ]:
%%bigquery
select last_update, length(split(split(last_update, ' ')[offset(0)], '/')[offset(2)]) as year_length
from
(select distinct last_update
from covid_19_modeled.Event_SQL1)
limit 12

In [ ]:
%%bigquery
select last_update, parse_datetime('%m/%d/%y %H:%M', last_update) as last_update_datetime
from
(select distinct last_update
from covid_19_modeled.Event_SQL1)
where length(split(split(last_update, ' ')[offset(0)], '/')[offset(2)]) = 2
limit 12

In [ ]:
%%bigquery
create or replace table covid_19_modeled.Event_SQL1 as
select location_id, parse_datetime('%m/%d/%y %H:%M', last_update) as last_update, confirmed, deaths, recovered
from covid_19_modeled.Event_SQL1
where length(split(split(last_update, ' ')[offset(0)], '/')[offset(2)]) = 2
union all
select location_id, parse_datetime('%m/%d/%Y %H:%M', last_update) as last_update, confirmed, deaths, recovered
from covid_19_modeled.Event_SQL1
where length(split(split(last_update, ' ')[offset(0)], '/')[offset(2)]) = 4

In [ ]:
%%bigquery
select * from
(select distinct last_update 
from covid_19_modeled.Event_SQL1)
order by last_update
limit 8

#### Task 6: Make new table Event_SQL3 as the union of Event_SQL1 + Event_SQL2

In [ ]:
%%bigquery
select count(*) total_event_count
from covid_19_modeled.Event_SQL3

### Option B: Standardize timestamps with Beam

#### Note: change kernel before running

In [ ]:
%run Event_beam1.py

### Check primary key constraint on Event table

In [ ]:
%%bigquery
select count(*) as total_event_count from covid_19_modeled.Event_Beam1

In [ ]:
%%bigquery
select count(*) as distinct_event_count from
(select distinct last_update, location_id as distinct_event_count from covid_19_modeled.Event_Beam1)

#### Q7: Find duplicate event records in Event_Beam1

In [ ]:
%%bigquery
select * from covid_19_modeled.Event_Beam1
where last_update = '2020-02-08 03:43:03'
and location_id = -2740713481135259291
limit 7

#### Note: we have the same duplicate records in Event_SQL3

### Option A: Detect and remove duplicate events with SQL

#### Q8: Remove duplicate events from Event_SQL3 with select distinct

In [ ]:
%%bigquery
select count(*) as total_event_count from covid_19_modeled.Event_SQL3

In [ ]:
%%bigquery
select location_id, last_update, count(*) as duplicate_events
from covid_19_modeled.Event_SQL3
group by location_id, last_update
having count(*) > 1
order by count(*) desc
limit 5

In [ ]:
%%bigquery
select * from covid_19_modeled.Event_SQL3
where location_id = -118082929383066870 and last_update = '2020-03-14 16:53:03'

In [ ]:
%%bigquery
select *, 
rank() over (partition by location_id, last_update order by confirmed desc) as rank
from covid_19_modeled.Event_SQL3
where location_id in (-118082929383066870, 5199822387082299175)
limit 30

In [ ]:
%%bigquery
create or replace table covid_19_modeled.Event_SQL4 as
select location_id, last_update, confirmed, deaths, recovered
from
(select *, 
rank() over (partition by location_id, last_update order by confirmed desc) as rank
from covid_19_modeled.Event_SQL3)
where rank = 1

#### Recheck primary key constraint on Event_SQL4

In [ ]:
%%bigquery
select count(*) total_event_count
from covid_19_modeled.Event_SQL4 

In [ ]:
%%bigquery
select count(*) distinct_event_count
from
(select distinct location_id, last_update
from covid_19_modeled.Event_SQL4) 

### Option B: Detect and remove duplicate event records with Beam

#### Remember to change kernel before running beam script

In [ ]:
%run Event_beam2.py

#### Recheck primary key constraint on Event table

In [ ]:
%%bigquery
select count(*) as total_event_count from covid_19_modeled.Event_Beam2

In [ ]:
%%bigquery
select count(*) as distinct_event_count from
(select distinct last_update, location_id as distinct_event_count from covid_19_modeled.Event_Beam2)

### Check that Event_Beam2 == Event_SQL4

In [ ]:
%%bigquery
select *, count(*) as duplicate_records
from
(select * 
from covid_19_modeled.Event_Beam2
union all
select * from 
covid_19_modeled.Event_SQL4)
group by location_id, last_update, confirmed, deaths, recovered
having count(*) != 2
order by location_id, last_update

In [ ]:
%%bigquery
(select * 
from covid_19_modeled.Event_Beam2
union distinct
select * from 
covid_19_modeled.Event_SQL4)
except distinct
(select * from covid_19_modeled.Event_Beam2
intersect distinct
select * from covid_19_modeled.Event_SQL4)

In [ ]:
%%bigquery
create or replace table covid_19_modeled.Event as
select * from covid_19_modeled.Event_SQL4

### Done with Event table. Do the same thing with Location table.  

### Option A: Standardize city, state with SQL

In [ ]:
%%bigquery
select count(*) location_count
from covid_19_modeled.Location_Temp

In [ ]:
%%bigquery
select id, state, strpos(state, ',') as index, country, latitude, longitude
from covid_19_modeled.Location_Temp
where strpos(state, ',') > 0
limit 6

In [ ]:
%%bigquery
select state as orig_state, split(state, ',')[offset(0)] parsed_state, split(state, ',')[offset(1)] parsed_city,
from covid_19_modeled.Location_Temp
where strpos(state, ',') > 0
limit 10

In [ ]:
%%bigquery
create or replace table covid_19_modeled.Location_SQL1 as
select id, split(state, ',')[offset(0)] city, split(state, ',')[offset(1)] state, country, latitude, longitude
from covid_19_modeled.Location_Temp
where strpos(state, ',') > 0

In [ ]:
%%bigquery
select count(*) as city_state_location_count
from covid_19_modeled.Location_SQL1

In [ ]:
%%bigquery
select id, state, cast(null as string) as city, country, latitude, longitude
from covid_19_modeled.Location_Temp
where strpos(state, ',') = 0
limit 6

In [ ]:
%%bigquery
create or replace table covid_19_modeled.Location_SQL2 as
select id, state, cast(null as string) as city, country, latitude, longitude
from covid_19_modeled.Location_Temp
where strpos(state, ',') = 0

In [ ]:
%%bigquery
select count(*) as state_location_count
from covid_19_modeled.Location_SQL2

In [ ]:
%%bigquery
select count(*) null_state_location_count
from covid_19_modeled.Location_Temp
where state is null

#### Q9: Create table Location_SQL3 based on Location_SQL1 + Location_SQL2 + null states

In [ ]:
%%bigquery
select count(*) as location_count
from covid_19_modeled.Location_SQL3

### Option B: Standardize city, state with Beam

In [ ]:
%run Location_beam1.py

In [ ]:
%%bigquery
select count(*) as total_location_count from covid_19_modeled.Location_Beam1

#### Check primary key constraint on Location table

In [ ]:
%%bigquery
select count(distinct id) as distinct_location_count from covid_19_modeled.Location_SQL3

In [ ]:
%%bigquery
select count(distinct id) as distinct_location_count from covid_19_modeled.Location_Beam1

In [ ]:
%%bigquery
select id, count(*) as duplicate_records
from covid_19_modeled.Location_Beam1
group by id
having count(*) > 1
limit 12

In [ ]:
%%bigquery
select * from covid_19_modeled.Location_Beam1
where id in (-823610271364515484, -5175654635300698393, 7752613693761280071)
order by id, latitude desc, longitude desc

### Option A: Remove duplicate location records with SQL

In [ ]:
%%bigquery
select *, 
rank() over (partition by id order by latitude desc, longitude desc) as rank
from covid_19_modeled.Location_SQL3
where id in (-823610271364515484, -5175654635300698393, 7752613693761280071)

In [ ]:
%%bigquery
create or replace table covid_19_modeled.Location_SQL4 as
select id, city, state, country, latitude, longitude from
(select *, 
rank() over (partition by id order by latitude desc, longitude desc) as rank
from covid_19_modeled.Location_SQL3)
where rank = 1

#### Recheck primary key on Location_SQL4

In [ ]:
%%bigquery
select count(*) as total_location_count from covid_19_modeled.Location_SQL4

In [ ]:
%%bigquery
select count(distinct id) as distinct_location_count from covid_19_modeled.Location_SQL4

### Option B: Remove duplicate location records with Beam 

In [ ]:
%run Location_beam2.py

### Recheck primary key on the Location table

In [ ]:
%%bigquery
select count(*) as total_location_count from covid_19_modeled.Location_Beam2

In [ ]:
%%bigquery
select count(distinct id) as distinct_location_count from covid_19_modeled.Location_Beam2

### Check foreign key (location_id on Event_Beam2)

In [ ]:
%%bigquery
select count(*) as foreign_key_violations
from covid_19_modeled.Event e left join covid_19_modeled.Location_Beam2 l on e.location_id = l.id
where l.id is null

In [ ]:
%%bigquery
select count(*) as foreign_key_violations
from covid_19_modeled.Event e left join covid_19_modeled.Location_SQL4 l on e.location_id = l.id
where l.id is null

### Check that Location_Beam2 == Location_SQL4 

In [ ]:
%%bigquery
select *, count(*) as count
from
(select *
from covid_19_modeled.Location_SQL4 
union all
select *
from covid_19_modeled.Location_Beam2)
group by id, city, state, country, latitude, longitude
having count(*) != 2
order by id

In [ ]:
%%bigquery
select * 
from covid_19_modeled.Location_SQL4
where id = -7197486485537661273

In [ ]:
%%bigquery
select * 
from covid_19_modeled.Location_Beam2
where id = -7197486485537661273

In [ ]:
%%bigquery
select * 
from covid_19_modeled.Location_Temp
where id = -7197486485537661273

### Manually looked up coordinates for French Polynesia here: https://latitude.to/map/pf/french-polynesia

In [ ]:
%%bigquery
update covid_19_modeled.Location_SQL4 set longitude = -149.4068 where id = -7197486485537661273

In [ ]:
%%bigquery
select *, count(*) as count
from
(select *
from covid_19_modeled.Location_SQL4 
union all
select *
from covid_19_modeled.Location_Beam2)
group by id, city, state, country, latitude, longitude
having count(*) != 2
order by id

In [ ]:
%%bigquery
(select * 
from covid_19_modeled.Location_Beam2
union distinct
select * from 
covid_19_modeled.Location_SQL4)
except distinct
(select * from covid_19_modeled.Location_Beam2
intersect distinct
select * from covid_19_modeled.Location_SQL4)

#### Q10: Create Location table from Location_SQL4

### Done modeling Event and Location tables. 